In [1]:
from skforecast.datasets import fetch_dataset
data = fetch_dataset(name='h2o_exog', raw=True, verbose=False)


In [1]:
import plotly.express as px
df = px.data.gapminder()


In [5]:
list(df.columns.values) + ["test"]

['country',
 'continent',
 'year',
 'lifeExp',
 'pop',
 'gdpPercap',
 'iso_alpha',
 'iso_num',
 'test']

In [9]:
import pandas as pd
data_test = pd.read_csv('data_test.csv').set_index("Datetime").sort_index()
data_test.index = pd.to_datetime(data_test.index)

In [10]:
import os
import joblib
cwd = os.path.abspath("")
modelling_file_path = os.path.join(cwd, './modelling/model/forecaster.joblib')

forecaster = joblib.load(modelling_file_path)
regressor_name = str(forecaster.regressor).split('()')[0]

In [11]:
exog_vars = ['month', 'dayofweek', 'hour', 'temperature_2m', 
             'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 
             'is_day', 'temperature_2m_rolling_mean_1_D', 'temperature_2m_rolling_max_1_D', 
             'temperature_2m_rolling_min_1_D', 'temperature_2m_rolling_mean_7_D', 
             'temperature_2m_rolling_max_7_D', 'temperature_2m_rolling_min_7_D']
steps = 24

In [12]:
data_test[exog_vars].index

DatetimeIndex(['2023-03-26 00:00:00+00:00', '2023-03-26 01:00:00+00:00',
               '2023-03-26 02:00:00+00:00', '2023-03-26 03:00:00+00:00',
               '2023-03-26 04:00:00+00:00', '2023-03-26 05:00:00+00:00',
               '2023-03-26 06:00:00+00:00', '2023-03-26 07:00:00+00:00',
               '2023-03-26 08:00:00+00:00', '2023-03-26 09:00:00+00:00',
               ...
               '2023-06-23 14:00:00+00:00', '2023-06-23 15:00:00+00:00',
               '2023-06-23 16:00:00+00:00', '2023-06-23 17:00:00+00:00',
               '2023-06-23 18:00:00+00:00', '2023-06-23 19:00:00+00:00',
               '2023-06-23 20:00:00+00:00', '2023-06-23 21:00:00+00:00',
               '2023-06-23 22:00:00+00:00', '2023-06-23 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Datetime', length=2160, freq=None)

In [15]:
data_test[exog_vars].index

DatetimeIndex(['2023-03-26 00:00:00+00:00', '2023-03-26 01:00:00+00:00',
               '2023-03-26 02:00:00+00:00', '2023-03-26 03:00:00+00:00',
               '2023-03-26 04:00:00+00:00', '2023-03-26 05:00:00+00:00',
               '2023-03-26 06:00:00+00:00', '2023-03-26 07:00:00+00:00',
               '2023-03-26 08:00:00+00:00', '2023-03-26 09:00:00+00:00',
               ...
               '2023-06-23 14:00:00+00:00', '2023-06-23 15:00:00+00:00',
               '2023-06-23 16:00:00+00:00', '2023-06-23 17:00:00+00:00',
               '2023-06-23 18:00:00+00:00', '2023-06-23 19:00:00+00:00',
               '2023-06-23 20:00:00+00:00', '2023-06-23 21:00:00+00:00',
               '2023-06-23 22:00:00+00:00', '2023-06-23 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Datetime', length=2160, freq=None)

In [14]:
predictions = forecaster.predict(steps = steps,
                                exog  = data_test[exog_vars])

c:\Users\alfon\AppData\Local\Programs\Python\Python312\Lib\site-packages\skforecast\utils\utils.py:1182: UserWarning: `exog` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(


TypeError: Expected index of type <class 'pandas.core.indexes.datetimes.DatetimeIndex'> for `exog`. Got <class 'pandas.core.indexes.range.RangeIndex'>.

In [1]:
import numpy as np
import pandas as pd
import os
import itertools

from copy import deepcopy
from utils import DataCleaningService, FeatureEngineering
from create_experiment import CreateExperiment
from get_exog_data import OpenMeteoApi
from sklearn.preprocessing import PolynomialFeatures

cwd = os.path.abspath("")
raw_data_file_path = os.path.join(cwd, '../data/power_load_BE_elia_15M_2015_2024.csv')
df = pd.read_csv(raw_data_file_path, delimiter = ";", on_bad_lines="skip")

In [18]:
cwd = os.path.abspath("")
modelling_file_path = os.path.join(cwd, './modelling/model/forecaster.joblib')
print(modelling_file_path)
with open(modelling_file_path, 'w') as file:
    file.write("test")

c:\Tommy\Work\Scripting\Portfolio\demand-forecast\src\./modelling/model/forecaster.joblib


In [7]:
clean_start_date = "2021-11-01"
clean_end_date   = "2023-06-23"
hour_in_week     = 24*7

# CLEAN INPUT DATA
cleaning_service = DataCleaningService()
df_clean = cleaning_service.normalize_to_timeseries_df(df = df,
                                          start_date = clean_start_date,
                                          end_date   = clean_end_date,
                                          date_index = "Datetime",)


In [12]:
date_index = "Datetime"
non_float_int_columns = ["Datetime", "Resolution code"]
[date_index] + non_float_int_columns

['Datetime', 'Datetime', 'Resolution code']

In [1]:
import pandas as pd
import os
from data_processing import steps, data, data_train, data_test, exog_vars, hour_in_week

cwd = os.path.abspath("")
backtest_result_file_path = os.path.join(cwd, '../data/backtest_result.csv')

df = pd.read_csv(backtest_result_file_path)

c:\Users\alfon\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total Load
Most recent forecast
Most recent P10
Most recent P90
Day-ahead 6PM forecast
Day-ahead 6PM P10
Day-ahead 6PM P90
Week-ahead forecast
Train dates : 2022-05-25 00:00:00+00:00 --- 2023-04-20 23:00:00+00:00  (n=7944)
Test dates  : 2023-04-21 00:00:00+00:00 --- 2023-06-15 23:00:00+00:00  (n=1344)


In [2]:
import joblib
modelling_file_path = os.path.join(cwd, './modelling/model/forecaster.joblib')
backtest_result_file_path = os.path.join(cwd, '../data/backtest_result.csv')

forecaster = joblib.load(modelling_file_path)
regressor_name = str(forecaster.regressor).split('()')[0]

In [3]:
from forecast import BacktestingForecaster 

In [4]:
bf = BacktestingForecaster(forecaster         = forecaster,
                          y                  = data['y'],
                          steps              = hour_in_week,
                          metric             = 'mean_absolute_percentage_error',
                          initial_train_size = len(data_train),
                          refit              = True,)
backtest_indexes = bf._create_backtesting_folds()

backtesting_folds_list = bf.get_backtesting_folds(backtest_indexes = backtest_indexes,
                         backtest_result_df = df,
                         regressor_name= regressor_name)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 7776
Number of observations used for backtesting: 1344
    Number of folds: 8
    Number of steps per fold: 168
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2022-06-01 00:00:00+00:00 -- 2023-04-20 23:00:00+00:00  (n=7776)
    Validation: 2023-04-21 00:00:00+00:00 -- 2023-04-27 23:00:00+00:00  (n=168)
Fold: 1
    Training:   2022-06-08 00:00:00+00:00 -- 2023-04-27 23:00:00+00:00  (n=7776)
    Validation: 2023-04-28 00:00:00+00:00 -- 2023-05-04 23:00:00+00:00  (n=168)
Fold: 2
    Training:   2022-06-15 00:00:00+00:00 -- 2023-05-04 23:00:00+00:00  (n=7776)
    Validation: 2023-05-05 00:00:00+00:00 -- 2023-05-11 23:00:00+00:00  (n=168)
Fold: 3
    Training:   2022-06-22 00:00:00+00:00 -- 2023-05-11 23:00:00+00:00  (n=7776)
    Validation: 2023-05-12 00:00:00+00:00 -- 2023-05-18 23:00:00+00:00  (n=168)
Fold: 4


In [7]:
backtesting_folds_list[1]

,y,"RandomForestRegressor(max_depth=10, n_estimators=50)"


In [61]:
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.ensemble import RandomForestRegressor

# Load your data
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv'
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]

# Split data into training and backtesting sets
n_backtest = 36 * 3  # Last 9 years for backtesting
data_train = data[:-n_backtest]
data_backtest = data[-n_backtest:]

# Initialize the forecaster
forecaster = ForecasterAutoreg(
    regressor=RandomForestRegressor(random_state=123),
    lags=15
)

# Perform backtesting
metric, predictions_backtest = backtesting_forecaster(
    forecaster=forecaster,
    y=data['y'],
    initial_train_size=len(data_train),
    steps=10,
    metric='mean_squared_error',
    refit=True,
    verbose=True
)

# Print fold information
print(f"Number of folds: {len(predictions_backtest)}")
for i, fold in enumerate(predictions_backtest):
    print(f"Fold {i}: {fold}")


Information of backtesting process
----------------------------------
Number of observations used for initial training: 96
Number of observations used for backtesting: 108
    Number of folds: 11
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 8 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 1999-06-01 00:00:00  (n=96)
    Validation: 1999-07-01 00:00:00 -- 2000-04-01 00:00:00  (n=10)
Fold: 1
    Training:   1992-05-01 00:00:00 -- 2000-04-01 00:00:00  (n=96)
    Validation: 2000-05-01 00:00:00 -- 2001-02-01 00:00:00  (n=10)
Fold: 2
    Training:   1993-03-01 00:00:00 -- 2001-02-01 00:00:00  (n=96)
    Validation: 2001-03-01 00:00:00 -- 2001-12-01 00:00:00  (n=10)
Fold: 3
    Training:   1994-01-01 00:00:00 -- 2001-12-01 00:00:00  (n=96)
    Validation: 2002-01-01 00:00:00 -- 2002-10-01 00:00:00  (n=10)
Fold: 4
    Training:   1994-11-01 00:00:00 -- 2002-10-01 00:00:00  (n=96)


100%|██████████| 11/11 [00:00<00:00, 274.42it/s]


Number of folds: 108
Fold 0: pred


In [2]:
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.ensemble import RandomForestRegressor
from forecast import BacktestingForecaster

# Load your data
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv'
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]

# Split data into training and backtesting sets
n_backtest = 36 * 3  # Last 9 years for backtesting
data_train = data[:-n_backtest]
data_backtest = data[-n_backtest:]

# Initialize the forecaster
forecaster = ForecasterAutoreg(
    regressor=RandomForestRegressor(random_state=123),
    lags=15
)
bf = BacktestingForecaster(
    forecaster=forecaster,
    y=data['y'],
    initial_train_size=len(data_train),
    steps=10,
    metric='mean_squared_error',
)


IndentationError: expected an indented block after 'if' statement on line 53 (forecast.py, line 54)

In [2]:
fold_training_index, fold_validation_index = bf._create_backtesting_folds()

In [3]:
fold_validation_index

[[Timestamp('1999-07-01 00:00:00'), Timestamp('1999-07-01 00:00:00')],
 [Timestamp('2000-05-01 00:00:00'), Timestamp('2000-05-01 00:00:00')],
 [Timestamp('2001-03-01 00:00:00'), Timestamp('2001-03-01 00:00:00')],
 [Timestamp('2002-01-01 00:00:00'), Timestamp('2002-01-01 00:00:00')],
 [Timestamp('2002-11-01 00:00:00'), Timestamp('2002-11-01 00:00:00')],
 [Timestamp('2003-09-01 00:00:00'), Timestamp('2003-09-01 00:00:00')],
 [Timestamp('2004-07-01 00:00:00'), Timestamp('2004-07-01 00:00:00')],
 [Timestamp('2005-05-01 00:00:00'), Timestamp('2005-05-01 00:00:00')],
 [Timestamp('2006-03-01 00:00:00'), Timestamp('2006-03-01 00:00:00')],
 [Timestamp('2007-01-01 00:00:00'), Timestamp('2007-01-01 00:00:00')],
 [Timestamp('2007-11-01 00:00:00'), Timestamp('2007-11-01 00:00:00')]]

In [3]:
# Perform backtesting
metric, predictions_backtest = backtesting_forecaster(
    forecaster=forecaster,
    y=data['y'],
    initial_train_size=len(data_train),
    steps=10,
    metric='mean_squared_error',
    refit=True,
    verbose=True
)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 96
Number of observations used for backtesting: 108
    Number of folds: 11
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 8 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 1999-06-01 00:00:00  (n=96)
    Validation: 1999-07-01 00:00:00 -- 2000-04-01 00:00:00  (n=10)
Fold: 1
    Training:   1992-05-01 00:00:00 -- 2000-04-01 00:00:00  (n=96)
    Validation: 2000-05-01 00:00:00 -- 2001-02-01 00:00:00  (n=10)
Fold: 2
    Training:   1993-03-01 00:00:00 -- 2001-02-01 00:00:00  (n=96)
    Validation: 2001-03-01 00:00:00 -- 2001-12-01 00:00:00  (n=10)
Fold: 3
    Training:   1994-01-01 00:00:00 -- 2001-12-01 00:00:00  (n=96)
    Validation: 2002-01-01 00:00:00 -- 2002-10-01 00:00:00  (n=10)
Fold: 4
    Training:   1994-11-01 00:00:00 -- 2002-10-01 00:00:00  (n=96)


100%|██████████| 11/11 [00:00<00:00, 300.68it/s]


In [41]:
"""
   folds = _create_backtesting_folds(
                data                  = y,
                window_size           = window_size,
                initial_train_size    = initial_train_size,
                test_size             = steps,
                externally_fitted     = externally_fitted,
                refit                 = refit,
                fixed_train_size      = fixed_train_size,
                gap                   = gap,
                skip_folds            = skip_folds,
                allow_incomplete_fold = allow_incomplete_fold,
                return_all_indexes    = False,
                differentiation       = differentiation,
                verbose               = verbose
                forecaster.window_size_diff

"""

'\n   folds = _create_backtesting_folds(\n                data                  = y,\n                window_size           = window_size,\n                initial_train_size    = initial_train_size,\n                test_size             = steps,\n                externally_fitted     = externally_fitted,\n                refit                 = refit,\n                fixed_train_size      = fixed_train_size,\n                gap                   = gap,\n                skip_folds            = skip_folds,\n                allow_incomplete_fold = allow_incomplete_fold,\n                return_all_indexes    = False,\n                differentiation       = differentiation,\n                verbose               = verbose\n\n'

In [4]:
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from sklearn.ensemble import RandomForestRegressor

# Load your data
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv'
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]

# Split data into training and backtesting sets
n_backtest = 36 * 3  # Last 9 years for backtesting
data_train = data[:-n_backtest]
data_backtest = data[-n_backtest:]

# Initialize the forecaster
forecaster = ForecasterAutoreg(
    regressor=RandomForestRegressor(random_state=123),
    lags=15
)

In [5]:
from skforecast.model_selection import _create_backtesting_folds

backtesting_folds = _create_backtesting_folds(
    data =data['y'],
    initial_train_size=len(data_train),
    refit=True,
    test_size= 10,
    window_size=forecaster.window_size_diff,

)



Information of backtesting process
----------------------------------
Number of observations used for initial training: 96
Number of observations used for backtesting: 108
    Number of folds: 11
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 8 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 1999-06-01 00:00:00  (n=96)
    Validation: 1999-07-01 00:00:00 -- 2000-04-01 00:00:00  (n=10)
Fold: 1
    Training:   1992-05-01 00:00:00 -- 2000-04-01 00:00:00  (n=96)
    Validation: 2000-05-01 00:00:00 -- 2001-02-01 00:00:00  (n=10)
Fold: 2
    Training:   1993-03-01 00:00:00 -- 2001-02-01 00:00:00  (n=96)
    Validation: 2001-03-01 00:00:00 -- 2001-12-01 00:00:00  (n=10)
Fold: 3
    Training:   1994-01-01 00:00:00 -- 2001-12-01 00:00:00  (n=96)
    Validation: 2002-01-01 00:00:00 -- 2002-10-01 00:00:00  (n=10)
Fold: 4
    Training:   1994-11-01 00:00:00 -- 2002-10-01 00:00:00  (n=96)


In [137]:
df

,0,1
0,"[0, 96]","[96, 106]"
1,"[10, 106]","[106, 116]"
2,"[20, 116]","[116, 126]"
3,"[30, 126]","[126, 136]"
4,"[40, 136]","[136, 146]"
5,"[50, 146]","[146, 156]"
6,"[60, 156]","[156, 166]"
7,"[70, 166]","[166, 176]"
8,"[80, 176]","[176, 186]"
9,"[90, 186]","[186, 196]"


In [106]:
df = pd.DataFrame([[n[0], n[3]] for n in backtesting_folds]) 

index_to_suffix_dict = {0 : "_start",
                        1 : "_end"}


def map_training_validation_indexes(df: pd.DataFrame, index_to_suffix_dict: dict) -> pd.DataFrame:
    df = pd.concat([pd.DataFrame
                        (df[index].to_list(), 
                            columns=['training'  + index_to_suffix_dict[index],
                                     'validation'+ index_to_suffix_dict[index]]) \
                                     for index in index_to_suffix_dict], axis = 1)
    return df

def readjust_end_indexes(df: pd.DataFrame) -> pd.DataFrame:
    df[["training_end", "validation_end"]] = df[["training_end", "validation_end"]].apply(lambda x : (x - 1))
    return df

df_train_val_index_transformed = (df.pipe(map_training_validation_indexes, index_to_suffix_dict)
                                  .pipe(readjust_end_indexes))   

In [142]:
def get_training_validation_folds(df: pd.DataFrame, df_input: pd.DataFrame, df_pred: pd.DataFrame, folds_type: str)-> list:
    columns = df.filter(regex = folds_type).columns
    column_start, column_end = [col for col in columns if "start" in col], [col for col in columns if "end" in col]
    folds_list = []
    for fold in df.index:
        validation_start = df[column_start].loc[fold].values[0]
        validation_end   = df[column_end].loc[fold].values[0]
        df_input_ = df_input.iloc[validation_start:validation_end]
        date_index = df_input_.index
        df_pred_ = df_pred.loc[date_index]
        validation_fold = pd.concat([df_input_, df_pred_], axis = 1)
        folds_list.append(validation_fold)
    return folds_list

In [143]:
folds_list_fin = get_training_validation_folds(df = df_train_val_index_transformed,
                              df_input = data,
                              df_pred= predictions_backtest,
                              folds_type= "validation") 

In [146]:
cwd = os.path.dirname(__file__)
backtest_result_file_path = os.path.join(cwd, '../data/backtest_result.csv')
backtest_result_df = pd.read_csv(backtest_result_file_path, index = False)

NameError: name 'os' is not defined